In [3]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from backtesting import Backtest, Strategy

path = '/Users/isaiahtp/Desktop/Zeppelin/z-algo/historical_testing/data/'


def in_reg_hours(date):
       return 9 * 60 + 30 <= date.hour * 60 + date.minute <= 16 * 60 + 1

symbol = path + "AAPL_2w_5sec.csv"
dia = pd.read_csv(symbol, parse_dates=[0], date_format='%Y-%m-%d %H:%M:%S%z')
dia.columns = map(str.capitalize, dia.columns)  # capitlize columns
dia['Date'] = dia['Date'].dt.tz_convert('America/New_York').dt.tz_localize(None)
#dia['Date'] = pd.to_datetime(dia['Date'], format='%Y-%m-%d %H:%M:%S')
dia = dia[dia["Date"].map(lambda x: in_reg_hours(x))]
dia.set_index('Date', inplace=True)
dia['Pct Change 1m'] = dia['Close'].pct_change(freq=pd.DateOffset(minutes=1))
dia['Pct Change 2m'] = dia['Close'].pct_change(freq=pd.DateOffset(minutes=2))

In [ ]:
for file in os.listdir(path):
    print(file)

In [ ]:
dia['Log Pct Change'] = np.log(dia.Open / dia.Open.shift())

In [ ]:
dia['Pct Change'].dropna().sample(len(dia), replace= True, ignore_index=True)

If the stock drops from between 0.5% and 0.9% in one minute, buy and sell at a 0.25% gain 

If the stock drops 2% in 2 minutes or less buy and place a sell order for 1.2% above 

If DOW drops 1.2% or more in the first 90 mins of the day, buy and sell for 0.6% gain 

If the Dow drops 175 points or more from its opening that day and then goes up more than 20 points and returns to within 15 points of that low, buy and sell for a 50 point gain 

In [ ]:
dia = pd.read_csv('dia_30d_5sec.csv')

In [ ]:
path = 'data/'
symbol = path + "DIA_30d_5sec.csv"
dia = pd.read_csv(symbol, parse_dates=[0], date_format='%Y-%m-%d %H:%M:%S%z')
dia.columns = map(str.capitalize, dia.columns)
dia['Date'] = dia['Date'].dt.tz_convert('America/New_York').dt.tz_localize(None)
# dia['Date'] = pd.to_datetime(dia['Date'], format='%Y-%m-%d %H:%M:%S')
dia.set_index('Date', inplace=True)
dia['Pct Change 1m'] = dia['Close'].pct_change(periods=12)
dia['Pct Change 2m'] = dia['Close'].pct_change(periods=24)

In [ ]:
dia

In [ ]:
from decimal import Decimal

all_ticks = pd.read_csv("../historical_testing/all_tickers_data.csv")
index = ['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades']
decimals = ['Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Profit Factor', 'Expectancy [%]', 'SQN']
all_ticks.index = index
all_ticks = all_ticks.transpose().sort_index().reset_index(drop=False, names="Symbol")
all_ticks[decimals] = all_ticks[decimals].applymap(Decimal)
all_ticks.insert(all_ticks.columns.get_loc('Return (Ann.) [%]')+1, "Return Diff. [%]", all_ticks['Return [%]'] - all_ticks['Buy & Hold Return [%]'])
all_ticks
# should convert Start/End to datetime and set index to rangeindex
# have symbol as a column

In [ ]:
all_ticks.apply(lambda x: type(x[0]), axis=0)

In [ ]:
all_ticks

In [ ]:
type(all_ticks.iloc[0])

In [ ]:
all_ticks.to_parquet("ticks_test.parquet")

In [ ]:
all_ticks.to_hdf("ticks_test.h5", key='test')

In [ ]:
%%timeit pd.read_parquet("ticks_test.parquet")s

In [4]:
from historical_testing.main import Zeppelin
bt = Backtest(dia, Zeppelin, cash=100000, commission=0.002, trade_on_close=False, hedging=True)
msg = bt.run()
ticker_col = pd.Series(msg, name='AAPL')
print(ticker_col)

Start                                                   2024-02-05 09:30:00
End                                                     2024-02-16 14:29:55
Duration                                                   11 days 04:59:55
Exposure Time [%]                                                 81.959755
Equity Final [$]                                                97967.07925
Equity Peak [$]                                                100159.08396
Return [%]                                                        -2.032921
Buy & Hold Return [%]                                             -2.486556
Return (Ann.) [%]                                                -42.739262
Volatility (Ann.) [%]                                              3.215667
Sharpe Ratio                                                            0.0
Sortino Ratio                                                           0.0
Calmar Ratio                                                            0.0
Max. Drawdow

In [8]:
aapl = pd.DataFrame(index=["AAPL"], data=[ticker_col.values], columns=[ticker_col.index])

In [9]:
%store aapl

Stored 'aapl' (DataFrame)


In [ ]:
from sqlalchemy import types
column_dtypes = {
    'Symbol': types.VARCHAR(length=5),
    'Start': types.Date,
    'Ended': types.Date,
    'Duration': types.Interval,
    'Exposure Time [%]': types.Numeric(18, 2),
    'Equity Final [$]': types.Numeric(18, 2),
    'Equity Peak [$]': types.Numeric(18, 2),
    'Return [%]': types.Numeric(18, 2),
    'Buy & Hold Return [%]': types.Numeric(18, 6),
    'Return (Ann.) [%]': types.Numeric(18, 6),
    'Volatility (Ann.) [%]': types.Numeric(18, 6),
    'Sharpe Ratio': types.Numeric(18, 6),
    'Sortino Ratio': types.Numeric(18, 6),
    'Calmar Ratio': types.Numeric(18, 6),
    'Max. Drawdown [%]': types.Numeric(18, 6),
    'Avg. Drawdown [%]': types.Numeric(18, 6),
    'Max. Drawdown Duration': types.Interval,
    'Avg. Drawdown Duration': types.Interval,
    '# Trades': types.Integer,
    'Win Rate [%]': types.VARCHAR(length=32),
    #'Win Rate [%]': types.Numeric(18, 2),
    'Best Trade [%]': types.Numeric(18, 6),
    'Worst Trade [%]': types.Numeric(18, 6),
    'Avg. Trade [%]': types.Numeric(18, 6),
    'Max. Trade Duration': types.Interval,
    'Avg. Trade Duration': types.Interval,
    'Profit Factor': types.Numeric(18, 6),
    'Expectancy [%]': types.Numeric(18, 6),
    'SQN': types.Numeric(18, 6),
    '_strategy': types.Text,
    '_equity_curve': types.Text,
    '_trades': types.Text
}


In [ ]:
ticker_col = pd.Series(msg, name='AAPL')
ticker_col

In [6]:
import pandas as pd
test = pd.read_parquet("ticks_test.parquet")
test = test.applymap(lambda x: str(x) if isinstance(x, pd.Timedelta) else x)

In [7]:
test

,Symbol,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],...,Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,_strategy,_equity_curve,_trades
0,AMD,2024-01-05 08:30:15,2024-02-16 15:59:55,42 days 07:29:40,93.02754729505476,106842.07926000762,111176.62660000440,6.8420792600076235,28.481667652276760,68.6088990863983100,...,-5.9667838389464880,0.28670301824820220,22 days 06:24:45,1 days 03:25:01,2.86518202765893370,0.29078531013290670,34.049464241650160,Zeppelin,Equity DrawdownPct...,Size EntryBar ... ExitTime...
1,AMGN,2024-01-05 09:33:10,2024-02-16 15:59:55,42 days 06:26:45,91.51569571151498,90345.93723999712,104251.33561999604,-9.6540627600028820,-5.878055260361314,-57.3781327926072750,...,-13.9946815199569200,-1.09256499426969580,13 days 00:57:10,3 days 17:52:00,0.32011523407682885,-0.98963549859416980,-12.541963686021680,Zeppelin,Equity DrawdownPct...,Size EntryBar ... ExitTime ...
2,JPM,2024-01-05 09:30:00,2024-02-16 15:59:55,42 days 06:29:55,91.83019338295524,104399.53051000187,104643.94369000182,4.3995305100018740,4.232403795773408,43.5713497328958250,...,-0.7319583203987068,0.40760270012172306,18 days 23:57:55,2 days 23:39:42,19.98399804209658400,0.40911149727636730,58.964404048954130,Zeppelin,Equity DrawdownPct...,Size EntryBar ... ExitTime ...
3,LYG,2024-01-05 09:30:00,2024-02-16 15:59:55,42 days 06:29:55,98.89743589743590,99730.96141199896,100002.88334000019,-0.2690385880010435,-9.915611814346006,-2.1844623472802915,...,-11.5876131554318850,-8.81974771795835900,38 days 06:29:50,30 days 07:25:27,0.01060940958026909,-8.69282360464310000,-63.255726466366370,Zeppelin,Equity DrawdownPc...,Size EntryBar ... ExitTime ...
4,ORCL,2024-01-05 09:30:00,2024-02-16 15:59:55,42 days 06:29:55,94.47951895095845,100913.55652500116,106390.34710500184,0.9135565250011569,8.355243938066023,7.9383821309094715,...,-5.6908684117711665,0.06847020939917403,11 days 03:06:00,1 days 12:01:03,1.24386799832500630,0.07781686174999197,4.812039337148872,Zeppelin,Equity DrawdownPc...,Size EntryBar ... ExitTime...
5,QQQ,2024-01-05 09:30:00,2024-02-16 15:59:55,42 days 06:29:55,91.42450142450143,103198.97114500006,105014.98856000001,3.1989711450000610,8.636088404480766,30.2786013800156120,...,-1.9488466752735079,0.26286682357354874,13 days 23:40:40,2 days 10:48:27,2.98335616147673340,0.26455708771242800,23.456120431152870,Zeppelin,Equity DrawdownPc...,Size EntryBar ... ExitTime ...
6,UL,2024-01-05 09:30:00,2024-02-16 15:59:55,42 days 06:29:55,92.91880341880342,102660.20394499980,102969.45142500059,2.6602039449997970,4.257507198683670,24.6745561131754840,...,-1.2515667474342740,0.89001238086989790,25 days 01:05:55,7 days 20:02:11,7.01420791001093000,0.89896987453893180,51.221441301162190,Zeppelin,Equity DrawdownPc...,Size EntryBar ... ExitTime ...
7,V,2024-01-05 09:30:00,2024-02-16 15:59:55,42 days 06:29:55,90.58333333333334,102858.98070500026,103770.01468500037,2.8589807050002563,6.790644171779133,27.7477066564356000,...,-1.0877575619087976,0.28038557036762500,16 days 06:27:05,3 days 09:41:01,4.56592798278148050,0.28107658454138280,45.049109549512200,Zeppelin,Equity DrawdownPc...,Size EntryBar ... ExitTime ...


In [ ]:
test = pd.DataFrame(index=["AAPL"], data=[ticker_col.values], columns=[ticker_col.index])
test.reset_index(inplace=True, drop=False, names='Symbol')
test